In [24]:
%load_ext autoreload
%autoreload 2
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM
import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from RainPredictionMachine.data import clean_data_rpm
df = clean_data_rpm()
df = df.clean_data(1)
df.head()

,Chuva,Pres,Pres_max,Pres_min,Radiacao,Temp,Temp_orvalho,Temp_max,Temp_min,Temp_orvalho_max,...,Umid,Dir_vento,Rajada_vento,Vel_vento,Estaçao,Latitude,Longitude,Altitude,datahora,classe_chuva
0,0.0,957.9,957.9,957.1,0.0,21.7,21.7,23.7,21.7,23.7,...,100.0,130.5,3.75,2.10,TUPA,-21.927251,-50.490251,498.0,2021-01-01 00:00:00+00:00,nao chove
1,0.0,958.0,958.3,957.9,0.0,21.4,21.4,21.7,21.4,21.6,...,100.0,109.0,5.40,3.35,TUPA,-21.927251,-50.490251,498.0,2021-01-01 01:00:00+00:00,nao chove
2,0.0,957.4,958.0,957.4,0.0,21.2,21.2,21.5,21.2,21.5,...,100.0,160.5,3.20,1.40,TUPA,-21.927251,-50.490251,498.0,2021-01-01 02:00:00+00:00,nao chove
3,0.0,956.6,957.4,956.6,0.0,21.0,21.0,21.3,20.9,21.3,...,100.0,160.5,3.20,1.40,TUPA,-21.927251,-50.490251,498.0,2021-01-01 03:00:00+00:00,nao chove
4,0.0,957.0,957.0,956.6,0.0,21.3,21.3,21.3,21.0,21.3,...,100.0,160.5,3.20,1.40,TUPA,-21.927251,-50.490251,498.0,2021-01-01 04:00:00+00:00,nao chove


In [35]:
from RainPredictionMachine.Trainer import pipe_creator
X_train_t, X_test_t, y_train, y_test = pipe_creator(df)

/Users/isabelagaio/.pyenv/versions/3.8.12/envs/rain-prediction-machine/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [42]:
X_train_t.shape

(6132, 20)

In [43]:
X_test_t.shape

(2628, 20)

In [44]:
X_train_t1 = np.expand_dims(X_train_t, 2)

In [45]:
X_train_t1.shape

(6132, 20, 1)

In [46]:
X_test_t1 = np.expand_dims(X_test_t, 2)

In [47]:
X_test_t1.shape

(2628, 20, 1)

In [48]:
normalizer = Normalization()
normalizer.adapt(X_train_t)
normalizer(X_train_t)[0]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([-0.02212413, -0.2426613 , -0.08204413,  0.2588284 , -0.43079457,
       -0.42099532, -0.35879606, -0.7298995 , -0.41831166,  0.12383739,
       -0.39299867, -0.29459488,  0.04114917,  0.19796313,  0.15076439,
        0.27520627,  0.27488652, -0.41163638,  0.10881617, -0.0988449 ],
      dtype=float32)>

In [49]:
norm = Normalization()

model = Sequential()

model.add(norm)

model.add(LSTM(units=20, activation='tanh'))
model.add(Dense(10, activation="tanh"))
model.add(Dense(4, activation="softmax"))

model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop', 
                  metrics=['accuracy']) 

In [50]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_t1, y_train, batch_size=16, epochs=1000, verbose=1,
         validation_split=0.2,
          callbacks=[es])

Epoch 1/1000
307/307 [==============================] - 3s 5ms/step - loss: 0.3330 - accuracy: 0.9242 - val_loss: 0.1973 - val_accuracy: 0.9438
Epoch 2/1000
307/307 [==============================] - 1s 4ms/step - loss: 0.2033 - accuracy: 0.9376 - val_loss: 0.1583 - val_accuracy: 0.9487
Epoch 3/1000
307/307 [==============================] - 1s 5ms/step - loss: 0.1558 - accuracy: 0.9448 - val_loss: 0.1218 - val_accuracy: 0.9568
Epoch 4/1000
307/307 [==============================] - 1s 5ms/step - loss: 0.1257 - accuracy: 0.9627 - val_loss: 0.1246 - val_accuracy: 0.9568
Epoch 5/1000
307/307 [==============================] - 1s 5ms/step - loss: 0.1010 - accuracy: 0.9710 - val_loss: 0.0812 - val_accuracy: 0.9780
Epoch 6/1000
307/307 [==============================] - 1s 5ms/step - loss: 0.0820 - accuracy: 0.9764 - val_loss: 0.0667 - val_accuracy: 0.9788
Epoch 7/1000
307/307 [==============================] - 1s 4ms/step - loss: 0.0653 - accuracy: 0.9827 - val_loss: 0.0540 - val_accuracy:

Epoch 57/1000
307/307 [==============================] - 1s 5ms/step - loss: 8.1803e-04 - accuracy: 0.9998 - val_loss: 1.2011e-05 - val_accuracy: 1.0000
Epoch 58/1000
307/307 [==============================] - 1s 5ms/step - loss: 4.6517e-06 - accuracy: 1.0000 - val_loss: 7.8847e-06 - val_accuracy: 1.0000
Epoch 59/1000
307/307 [==============================] - 1s 5ms/step - loss: 0.0026 - accuracy: 0.9998 - val_loss: 6.6653e-06 - val_accuracy: 1.0000
Epoch 60/1000
307/307 [==============================] - 1s 5ms/step - loss: 3.2051e-06 - accuracy: 1.0000 - val_loss: 5.8223e-06 - val_accuracy: 1.0000
Epoch 61/1000
307/307 [==============================] - 1s 5ms/step - loss: 2.7875e-06 - accuracy: 1.0000 - val_loss: 5.2297e-06 - val_accuracy: 1.0000
Epoch 62/1000
307/307 [==============================] - 2s 5ms/step - loss: 2.3867e-06 - accuracy: 1.0000 - val_loss: 4.5591e-06 - val_accuracy: 1.0000
Epoch 63/1000
307/307 [==============================] - 1s 5ms/step - loss: 2.0595e-0

307/307 [==============================] - 1s 5ms/step - loss: 1.1641e-08 - accuracy: 1.0000 - val_loss: 2.2637e-08 - val_accuracy: 1.0000
Epoch 111/1000
307/307 [==============================] - 1s 4ms/step - loss: 1.0742e-08 - accuracy: 1.0000 - val_loss: 2.0888e-08 - val_accuracy: 1.0000
Epoch 112/1000
307/307 [==============================] - 1s 4ms/step - loss: 1.0013e-08 - accuracy: 1.0000 - val_loss: 1.9528e-08 - val_accuracy: 1.0000
Epoch 113/1000
307/307 [==============================] - 1s 4ms/step - loss: 9.2839e-09 - accuracy: 1.0000 - val_loss: 1.8168e-08 - val_accuracy: 1.0000
Epoch 114/1000
307/307 [==============================] - 1s 4ms/step - loss: 8.6763e-09 - accuracy: 1.0000 - val_loss: 1.7099e-08 - val_accuracy: 1.0000
Epoch 115/1000
307/307 [==============================] - 1s 4ms/step - loss: 8.1173e-09 - accuracy: 1.0000 - val_loss: 1.5933e-08 - val_accuracy: 1.0000
Epoch 116/1000
307/307 [==============================] - 1s 4ms/step - loss: 7.6070e-09 - 

307/307 [==============================] - 2s 5ms/step - loss: 9.4784e-10 - accuracy: 1.0000 - val_loss: 2.2346e-09 - val_accuracy: 1.0000
Epoch 164/1000
307/307 [==============================] - 2s 5ms/step - loss: 9.2354e-10 - accuracy: 1.0000 - val_loss: 2.1374e-09 - val_accuracy: 1.0000
Epoch 165/1000
307/307 [==============================] - 2s 5ms/step - loss: 8.7493e-10 - accuracy: 1.0000 - val_loss: 2.1374e-09 - val_accuracy: 1.0000
Epoch 166/1000
307/307 [==============================] - 2s 5ms/step - loss: 8.5063e-10 - accuracy: 1.0000 - val_loss: 2.1374e-09 - val_accuracy: 1.0000
Epoch 167/1000
307/307 [==============================] - 2s 5ms/step - loss: 8.5063e-10 - accuracy: 1.0000 - val_loss: 2.0403e-09 - val_accuracy: 1.0000
Epoch 168/1000
307/307 [==============================] - 1s 5ms/step - loss: 8.5063e-10 - accuracy: 1.0000 - val_loss: 2.0403e-09 - val_accuracy: 1.0000
Epoch 169/1000
307/307 [==============================] - 2s 5ms/step - loss: 8.5063e-10 - 